# LangChain QuickStart Using Google Gemini

根据 LangChain 官方的 [QuickStart](https://python.langchain.com/docs/get_started/quickstart/) 教程，在 Google Colab 中使用 Google Gemini API 学习 LangChain 基本操作。

*这里选择使用 Google Gemini 是因为只需要一个 Google 账号就能免费使用 Gemini API，而获取 OpenAI Key 不管什么途径都比较麻烦且需要付费。*

## Installation

In [ ]:
%pip install --upgrade --quiet langchain

以上命令会安装 LangChain 的最基本依赖。如果想要将 LangChain 与大模型、数据库等进行集成，还需要额外单独安装指定的依赖库。

## Building with LangChain


LangChain 使我们能够构建应用，这些应用能够把外部的数据源和计算资源接入到大型语言模型（LLM）。在本快速入门教程中，我们会介绍几种实现这一点的不同方法。首先，我们将介绍一个简单的 LLM 链。这个链条仅仅依赖于提示模板中的信息来给出反馈。其次，我们会构建一个检索链条，这个链条会从一个独立的数据库获取数据，并将其传送到提示模板中。之后，我们将加入聊天历史，创建一个支持对话的检索链条，让您能够以聊天的形式与 LLM 互动，并且 LLM 能够记住之前的提问。~~最终，我们将构建一个智能代理 —— 它会利用 LLM 判断在回答问题时是否需要去获取数据。~~

## LLM Chain

在这里使用 Google Gemini API，先安装 Google AI 的 Python 库：

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

从 Colab 中导入 Gemini API 密钥：

In [ ]:
from google.colab import userdata

API_KEY = userdata.get('API_KEY')

之后让我们初始化模型，同时因为 `gemini-1.5-pro-latest` 仅允许每分钟 2 次查询，每天最多 1000 次查询，而 `gemini-pro` 允许每分钟 60 个请求，无查询总数限制，所以这里先用 `gemini-pro`：

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=API_KEY)

我们来看看它对 LangSmith 是什么的解释 —— 这个概念在它的训练数据中不一定包含，因此我们可能不会得到一个很准确的答案。

In [ ]:
llm.invoke("how can langsmith help with testing?")

"**Test Case Generation:**\n\n* **Natural Language Input:** Allows testers to write test cases in natural language, which Langsmith then translates into executable test scripts. This simplifies test case creation and reduces the risk of errors.\n* **Automated Test Case Generation:** Uses AI algorithms to generate test cases based on requirements or existing code. This speeds up test case development and ensures comprehensive coverage.\n* **Exploratory Testing:** Supports exploratory testing by providing language-based prompts to guide testers in discovering potential defects.\n\n**Test Execution:**\n\n* **Test Runner:** Executes test cases automatically or manually and captures results.\n* **Parallel Execution:** Allows multiple test cases to be executed concurrently, reducing overall testing time.\n* **Reporting:** Generates detailed test reports that include pass/fail status, execution time, and any errors or defects encountered.\n\n**Test Management:**\n\n* **Test Planning:** Helps 

我们还可以使用提示模板来指导它给出回答。这些提示模板能够将用户的初始输入优化成更适合 LLM 的输入形式。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

现在我们可以将这些组合成一个简单的 LLM 链。

In [ ]:
chain = prompt | llm

我们现在可以调用它并询问同样的问题。尽管它依然不知道答案，但它应该以更适合技术撰稿人的语气来回应！

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

'**How Langsmith Can Assist with Testing**\n\n**1. Automated Test Case Generation:**\n\n* Generate comprehensive test cases from natural language requirements.\n* Support various testing methodologies (e.g., black-box, white-box).\n* Improve test coverage and reduce manual effort.\n\n**2. Test Case Management:**\n\n* Organize and manage test cases in a structured hierarchy.\n* Track test execution status and results.\n* Provide real-time visibility into testing progress.\n\n**3. Test Data Generation:**\n\n* Generate realistic and diverse test data based on specified constraints.\n* Ensure data validity and consistency.\n* Reduce the risk of data-related test failures.\n\n**4. Defect Reporting:**\n\n* Automatically generate defect reports based on test results.\n* Capture detailed information about defects, including steps to reproduce.\n* Facilitate efficient defect tracking and resolution.\n\n**5. Collaboration and Communication:**\n\n* Provide a central platform for testers, develope

ChatModel 的输出（也就是这个 LLM 链的输出结果）是一条消息格式的内容。不过，在很多情况下，将输出结果作为普通文本字符串处理会更加方便。让我们加入一个简单的输出转换器，将聊天消息格式的输出转换为文本字符串。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

我们现在可以将此添加到之前的链中：

In [ ]:
chain = prompt | llm | output_parser

我们现在可以调用它并提出同样的问题。答案现在将是一个字符串（而不是 ChatMessage）。

*当前使用 Gemini API 在 Colab 中这几次输出结果格式并没有什么区别，已经是字符串了。*

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

"**Langsmith's Testing Capabilities for Technical Documentation**\n\n**Automated Testing:**\n\n* **Syntax checking:** Verifies the correctness of code snippets and examples in the documentation.\n* **Link checking:** Ensures all links in the documentation are valid and lead to the intended destinations.\n* **Image verification:** Checks if images are displayed correctly and have appropriate alt text.\n* **Spell checking and grammar correction:** Identifies and corrects spelling and grammar errors.\n\n**Collaboration and Review:**\n\n* **Shared document editing:** Allows multiple stakeholders to collaborate on the documentation and track changes.\n* **Integrated commenting:** Facilitates discussions and feedback on specific sections of the documentation.\n* **Automated review tools:** Provides suggestions for clarity, conciseness, and adherence to style guidelines.\n\n**User Experience and Accessibility Testing:**\n\n* **Readability analysis:** Assesses the readability of the documentat

## Retrieval Chain

为了准确地回答这个问题（"how can langsmith help with testing?"），我们需要向大语言模型（LLM）补充更多相关的信息。这可以通过信息检索来完成。当你手头的**数据太多**，不能直接全部传给大语言模型时，信息检索就显得尤为重要。你可以利用一个信息检索工具来筛选出与问题最相关的信息片段，并只将这些信息片段输入到模型中。

在这个流程中，我们会从一个检索工具中找到相关的文档，接着把这些文档内容嵌入到提示语中。检索工具的后端可以是多种形式：比如 SQL 表格、网络资源等。但在本例中，我们打算创建一个向量存储空间，并将其作为检索工具。

首先，我们需要准备好我们计划建立索引的数据。为了完成这一步骤，我们会利用 WebBaseLoader 这个工具。进行这个操作之前，需要先确保我们的环境中安装了 BeautifulSoup 这个库。

In [ ]:
%pip install --upgrade --quiet beautifulsoup4

导入和使用 WebBaseLoader：

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")  # 404
loader = WebBaseLoader("https://web.archive.org/web/20240324062817/https://docs.smith.langchain.com/user_guide")

docs = loader.load()

接下来，我们需要把数据索引进向量数据库。此过程需要借助一些关键组件：一个嵌入模型以及一个向量数据库。

至于如何使用嵌入模型，这里还是使用 Gemini API 的方式进行：

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)

现在，我们可以使用这个嵌入模型来导入文档至一个向量数据库。为了方便起见，我们此处选用一个简易的本地向量数据库：Facebook AI Similarity Search (FAISS)。

首先，我们需要安装必要的软件包：

In [ ]:
%pip install --upgrade --quiet faiss-cpu

接下来，我们将创建我们的向量索引：

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

现在，我们已经将数据索引入向量数据库中，下一步是构建一个检索链。此链会处理传入的问题，搜索相关的文档，随后将这些文档连同原先的提问一起提交给 LLM，让它来回答原始的问题。

首先，我们来搭建一个链，这个链能够处理用户提出的问题并根据检索到的文档生成回答。

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

如果我们想的话，可以通过直接输入文档来亲自执行这一过程：

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can visualize test results.'

然而，我们希望从我们刚刚设置的检索系统中得到文档。这样，我们就可以利用这个系统动态地筛选出针对特定问题最相关的文档，并用于生成回答。

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

现在，我们可以调用该链。这将返回一个字典类型的数据 - LLM 提供的响应将在 `answer` 键下找到。

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
response

{'input': 'how can langsmith help with testing?',
 'context': [Document(page_content='LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3 captures\n16 Feb 2024 - 24 Mar 2024\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFeb\nMAR\nApr\n\n\n\n\n24\n\n\n\n\n2023\n2024\n2025\n\n\n\n\n\n\n\nsuccess\nfail\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n About this capture\n\n\n\n\n\n\nCOLLECTED BY\n\n\n\nCollection: Save Page Now Outlinks\n\n\n\n\nTIMESTAMPS\n\n\n\n\n\nThe Wayback Machine - https://web.archive.org/web/20240324062817/https://docs.smith.langchain.com/user_guide', metadata={'source': 'https://web.archive.org/web/20240324062817/https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each 

In [ ]:
print(response["answer"])

LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also makes it easy to run custom evaluations (both LLM and heuristic based) to score test results.


## Conversation Retrieval Chain

我们迄今为止构建的这个流程仅能回答单一问题。大语言模型的主流应用领域之一就是搭建聊天机器人。那么，我们应该如何改进这个流程，使它可以处理连续的对话问题呢？

我们可以继续使用 `create_retrieval_chain` 函数，但需要做出两方面的调整：

1. 我们的检索方法现在应综合考虑整个对话的历史信息，而不是只针对最近的一次提问。
2. 最终的 LLM 链，也需要将全部的历史信息纳入考虑。

为了改进检索过程，我们需创建一个新的链。这个链会处理最新的输入（input）与过往的对话内容（chat_history），并借助 LLM 生成搜索问题。

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

我们可以尝试一个场景，其中用户提出了一个后续问题，来验证这个流程的效果。

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response = retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
response

[Document(page_content='Skip to main content\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for this phase.Debugging‚ÄãWhen de

你会发现，这样做检索出了有关于 LangSmith 测试的文档。这是因为 LLM 根据聊天的历史内容和追加的问题，生成了一条新的搜索命令。

既然我们已经拥有了这个新的检索工具，我们可以基于这些得到的文档，构建一条新的流程，以此继续进行对话。

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

现在，我们可以进行一次完整的测试：

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
response

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe

In [ ]:
print(response['answer'])

LangSmith provides multiple ways to test your LLM applications:
- Initial Test Set: Create datasets with collections of inputs and reference outputs to run tests on your LLM applications.
- Comparison View: View results for different configurations on the same data points side-by-side to track and diagnose regressions in test scores across multiple revisions of your application.
- Playground: Quickly test out different prompts and models and compare different runs.
- Beta Testing: Collect more data on how your LLM applications are performing in real-world scenarios and develop an understanding of the types of inputs the app is performing well or poorly on.
- Capturing Feedback: Gather human feedback on the responses your application is producing to highlight edge cases that are causing problematic responses.
- Annotating Traces: Send runs to annotation queues, which allow annotators to closely inspect interesting traces and annotate them with respect to different criteria.
- Adding Run

我们可以看到，这一系列动作让我们得到了一个条理清晰的答案——这意味着，我们已经成功地把检索流程转化为了一个聊天机器人！

## Agent

使用 Gemini API 来创建 Agent 过程要比官方基于 OpenAI 的示例复杂，具体代码可以参考 [Gemini Agent Example](https://github.com/MikeChan-HK/Gemini-agent-example)，这里先暂时跳过。

## Serving with LangServe

暂时先不学习 LangSmith 和 LangServe，跳过。